# Releasing LM-Evaluation-Harness v0.4.0

With the vast amount of work done in the field today, it helps to have a tool that people can use easily to share their results and use to check others to ensure reported numbers are valid. The LM Evaluation Harness is one such tool the community has used extensively. We want to continue to support the community and with that in mind, we’re excited to announce a major update on the LM Evaluation Harness to further our goal for open and accessible AI research.

Our refactor stems from our desires to make the following believed best practices easier to carry out.  

1.   Never copy results from other papers
2.   Always share your exact prompts
3.   Always provide model outputs
4.   Qualitatively review a small batch of outputs before running evaluation jobs at scale

We also wanted to make the library a better experience to use and to contribute or design evaluations within. New features in the new release that serve this purpose include:

1. Faster Evaluation Runtimes (accelerated data-parallel inference with HF Transformers + Accelerate, and commonly used or faster inference libraries such as vLLM and Llama-CPP)
2. Easier addition and sharing of new tasks (YAML-based task config formats, allowing single-file sharing of custom tasks)
3. More configurability, for more advanced workflows and easier operation with modifying prompts
4. Better logging of data at runtime and post-hoc

In this notebook we will be going through a short tutorial on how things work.

## Install LM-Eval

In [1]:
# Install LM-Eval
!pip install git+https://github.com/EleutherAI/lm-evaluation-harness.git

^C


  Running command git clone --filter=blob:none --quiet https://github.com/EleutherAI/lm-evaluation-harness.git 'C:\Users\blacksun\AppData\Local\Temp\pip-req-build-7v95un34'


Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
  Cloning https://github.com/EleutherAI/lm-evaluation-harness.git to c:\users\blacksun\appdata\local\temp\pip-req-build-7v95un34
  Resolved https://github.com/EleutherAI/lm-evaluation-harness.git to commit ade1cc4e481c65f8be44b8f4c6fc2c4942e6bd33
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for lm_eval: filename=lm_eval-0.4.5-py3-none-any.whl size=2410938 sha256=f7f8b139a5e0f6310ef5dbcbfeba67157cbade569f3fb94c2e7ccda488d77793
  Stored in directory: C:\Users\blacksun\AppData\Local\Temp\pip-ephem-wheel-cache-jfyvgbf1\wheels\ff\d5\f4\949f2b285369fc332f8dbba43f220ebd88387ee2f2c0c1174f
Successfully built lm_eval
  Attempting uninstall:

## Create new evaluation tasks with config-based tasks

Even within the same task, many works have reported numbers based on different choices of evaluation. Some report on the test sets, validation sets, or even subset of the training sets. Others have specialized prompts and verbalizers. We introduce YAMLs to allow users to easily make different variations. By leveraging the YAML configs to configure evaluations, the refactored LM-Eval takes the methods of the `Task` object and makes them configurable by setting the appropriate attributes in the config file. There, users can set the tasks they want by setting the name of the HF dataset (local tasks are also possible), the dataset splits used, and much more. Key configurations relating to prompting, such as `doc_to_text`, previously implemented as a method of the same name, are now configurable with jinja2 to allow high-level scripting to transform a HF dataset to text string as input to the model.
即使在相同的任务中，许多研究工作也会根据不同的评估选择报告数据。例如，有些使用测试集、验证集，甚至训练集的子集进行评估；还有的使用特定的提示词和词汇表。为了解决这个问题，我们引入了 YAML 配置文件，让用户能够轻松实现不同的变体。通过利用 YAML 配置来配置评估，重构后的 LM-Eval 将 Task 对象的方法转换为可配置的属性，用户可以在配置文件中进行设置。在这里，用户可以通过设置所需的 Hugging Face 数据集名称（也可以使用本地任务）、数据集的拆分方式等内容来自定义任务。
与提示相关的关键配置，例如之前以同名方法实现的 doc_to_text，现在可以通过 jinja2 进行配置，以便进行高级脚本化，将 Hugging Face 数据集转换为模型输入的文本字符串。

A core-feature to LM-Eval is to configure tasks with YAML configs. With configs, you can fill preset fields to easily set up a task.
Here, we write a demo YAML config for a multiple-choice evaluation of BoolQ:
LM-Eval 的一个核心功能是使用 YAML 配置文件来配置任务。通过这些配置，你可以填充预设字段，轻松设置一个任务。
这里我们编写了一个用于 BoolQ 多项选择评估的 YAML 配置示例：

In [2]:
YAML_boolq_string = """
task: demo_boolq
dataset_path: super_glue
dataset_name: boolq
output_type: multiple_choice
training_split: train
validation_split: validation
doc_to_text: "{{passage}}\nQuestion: {{question}}?\nAnswer:"
doc_to_target: label
doc_to_choice: ["no", "yes"]
should_decontaminate: true
doc_to_decontamination_query: passage
metric_list:
  - metric: acc
"""
with open("boolq.yaml", "w") as f:
    f.write(YAML_boolq_string)

And we can now run evaluation on this task, by pointing to the config file we've just created:

In [ ]:
lm_eval --model hf --model_args pretrained=EleutherAI/pythia-14m --include_path ./ --tasks demo_boolq --limit 10 --device cpu

Often, tasks are part of a larger group used to measure different capabilities. The dynamism of the field today means new dimensions of evaluation can come about which would mix and match new and older tasks alike. In LM-Eval, We can also group tasks and call that the group name to evaluate on a set of tasks easily. In this instance, let's evaluate the tag `yes_or_no_tasks` which comprise of the tasks `demo_boolq` and `demo_cola`; tasks which are multiple choice tasks with options `yes` and `no` as the name suggests.

<!-- making new groups is easier than ever, allowing user to work bottom-up by makiing individual tasks and linking them to a group or Top-Down, making a new group by listing existing tasks.

We also show the aggregate across samples besides only showing the aggregation between subtasks. This may come in handy when certain groups want to be aggregated as a single task. -->




In [5]:
YAML_cola_string = """
tag: yes_or_no_tasks
task: demo_cola
dataset_path: glue
dataset_name: cola
output_type: multiple_choice
training_split: train
validation_split: validation
doc_to_text: "{{sentence}}\nQuestion: Does this sentence make sense?\nAnswer:"
doc_to_target: label
doc_to_choice: ["no", "yes"]
should_decontaminate: true
doc_to_decontamination_query: sentence
metric_list:
  - metric: acc
"""
with open("cola.yaml", "w") as f:
    f.write(YAML_cola_string)

In [6]:
# !accelerate launch --no_python
!lm_eval \
    --model hf \
    --model_args pretrained=EleutherAI/pythia-2.8b \
    --include_path ./ \
    --tasks yes_or_no_tasks \
    --limit 10 \
    --output output/yes_or_no_tasks/ \
    --log_samples

2023-11-29:11:56:33,016 INFO     [utils.py:160] NumExpr defaulting to 2 threads.
2023-11-29 11:56:33.852995: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-29 11:56:33.853050: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-29 11:56:33.853087: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-29 11:56:35.129047: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-11-29:11:56:38,546 INFO     [__main__.py:132] Verbosity set to INFO
2023-11-29:11:56:47,509 WARNING  [__main__.py:138]  --limit SHOULD ONLY BE USED FOR 

## Edit Prompt Templates Quickly

The following is a yaml made to evaluate the specific subtask of `high_school_geography` from MMLU. It uses the standard prompt where the we choose the letters from the options with most likelihood as the model's prediction.

In [7]:
YAML_mmlu_geo_string = """
task: demo_mmlu_high_school_geography
dataset_path: cais/mmlu
dataset_name: high_school_geography
description: "The following are multiple choice questions (with answers) about high school geography.\n\n"
test_split: test
fewshot_split: dev
fewshot_config:
  sampler: first_n
output_type: multiple_choice
doc_to_text: "{{question.strip()}}\nA. {{choices[0]}}\nB. {{choices[1]}}\nC. {{choices[2]}}\nD. {{choices[3]}}\nAnswer:"
doc_to_choice: ["A", "B", "C", "D"]
doc_to_target: answer
metric_list:
  - metric: acc
    aggregation: mean
    higher_is_better: true
  - metric: acc_norm
    aggregation: mean
    higher_is_better: true
"""
with open("mmlu_high_school_geography.yaml", "w") as f:
    f.write(YAML_mmlu_geo_string)

In [8]:
# !accelerate launch --no_python
!lm_eval \
    --model hf \
    --model_args pretrained=EleutherAI/pythia-2.8b \
    --include_path ./ \
    --tasks demo_mmlu_high_school_geography \
    --limit 10 \
    --output output/mmlu_high_school_geography/ \
    --log_samples

2023-11-29:11:57:23,598 INFO     [utils.py:160] NumExpr defaulting to 2 threads.
2023-11-29 11:57:24.719750: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-29 11:57:24.719806: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-29 11:57:24.719847: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-29 11:57:26.656125: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-11-29:11:57:31,563 INFO     [__main__.py:132] Verbosity set to INFO
2023-11-29:11:57:40,541 WARNING  [__main__.py:138]  --limit SHOULD ONLY BE USED FOR 

We could also evaluate this task in a different way. For example, instead of observing the loglikelihood of the letters, we can instead evaluate on the choices themselves as the continuation. This is done by simply changing `doc_to_choice` from a list of letters to the corresponding `choices` field from the HF dataset. We write `"{{choices}}"` so that the string field is interpreted as jinja string that acquires the list from the HF dataset directly.

Another convenient feature here is since we're only modifying the `doc_to_choice` and the rest of config is the same as the task above, we can use the above configuration as a template by using `include: mmlu_high_school_geography.yaml` to load the config from that file. We'll need to add a unique task name as to not colide with the existing yaml config we're including. For this case we'll simply name this one `mmlu_high_school_geography_continuation`. `doc_to_text` is added here just for sake of clarity.

In [9]:
YAML_mmlu_geo_string = """
include: mmlu_high_school_geography.yaml
task: demo_mmlu_high_school_geography_continuation
doc_to_text: "{{question.strip()}}\nA. {{choices[0]}}\nB. {{choices[1]}}\nC. {{choices[2]}}\nD. {{choices[3]}}\nAnswer:"
doc_to_choice: "{{choices}}"
"""
with open("mmlu_high_school_geography_continuation.yaml", "w") as f:
    f.write(YAML_mmlu_geo_string)

In [10]:
# !accelerate launch --no_python
!lm_eval \
    --model hf \
    --model_args pretrained=EleutherAI/pythia-2.8b \
    --include_path ./ \
    --tasks demo_mmlu_high_school_geography_continuation \
    --limit 10 \
    --output output/mmlu_high_school_geography_continuation/ \
    --log_samples

2023-11-29:11:58:21,284 INFO     [utils.py:160] NumExpr defaulting to 2 threads.
2023-11-29 11:58:22.850159: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-29 11:58:22.850219: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-29 11:58:22.850254: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-29 11:58:24.948103: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-11-29:11:58:28,460 INFO     [__main__.py:132] Verbosity set to INFO
2023-11-29:11:58:37,935 WARNING  [__main__.py:138]  --limit SHOULD ONLY BE USED FOR 

If we take a look at the samples, we can see that it is in fact evaluating the continuation based on the choices rather than the letters.

In [11]:
from google.colab import files


files.view(
    "output/mmlu_high_school_geography_continuation/pretrained__EleutherAI__pythia-2.8b_demo_mmlu_high_school_geography_continuation.jsonl"
)

<IPython.core.display.Javascript object>

## Closer Look at YAML Fields

To prepare a task we can simply fill in a YAML config with the relevant information.

`output_type`
The current provided evaluation types comprise of the following:
1.   `loglikelihood`: Evaluates the loglikelihood of a continuation, conditioned on some input string.
2.   `loglikelihood_rolling`: evaluate the loglikelihood of producing a string, conditioned on the empty string. (Used for perplexity evaluations)
3.   `multiple_choice`: Evaluates loglikelihood among the a number of choices predicted by the model.
4.   `greedy_until`: Model outputs greedy generation (can be configured to to use beam search and other generation-related parameters)

The core prompt revolves around 3 fields.
1. `doc_to_text`: Denotes the prompt template that will be used as input to the model.
2. `doc_to_choice`: Available choices that will be used as continuation for the model. This is used when the `output_type` is `multiple_choice`, and otherwise can be left as `None`.
3. `doc_to_target`: When `output_type` is `multiple_choice`, this can be an index that corresponds to the correct answer, or the answer string itself (must be a subset of `doc_to_choice`). For other tasks, this is expected to be a string. You can fill this field with a feature name from the HF dataset so long as the resulting feature follows the conditioned described.

These three fields can be expressed as strings, column names from the source dataset, or as Jinja2 templates that can use fields from the source dataset as variables.


## What if Jinja is not Sufficient?

There can be times where the Jinja2 templating language is not enough to make the prompt we had in mind. There are a few ways to circumvent this limitation:

1. Use `!function` operator for the prompt-related fields to pass a python function that takes as input the dataset row, and will output the prompt template component.
2. Perform a transformation on the dataset beforehand.

Below, we show an example of using `!function` to create `doc_to_text` from a python function:

In [12]:
YAML_mmlu_geo_string = """
include: mmlu_high_school_geography.yaml
task: demo_mmlu_high_school_geography_function_prompt
doc_to_text: !function utils.doc_to_text
doc_to_choice: "{{choices}}"
"""
with open("demo_mmlu_high_school_geography_function_prompt.yaml", "w") as f:
    f.write(YAML_mmlu_geo_string)

DOC_TO_TEXT = """
def doc_to_text(x):
    question = x["question"].strip()
    choices = x["choices"]
    option_a = choices[0]
    option_b = choices[1]
    option_c = choices[2]
    option_d = choices[3]
    return f"{question}\\nA. {option_a}\\nB. {option_b}\\nC. {option_c}\\nD. {option_d}\\nAnswer:"
"""
with open("utils.py", "w") as f:
    f.write(DOC_TO_TEXT)

!lm_eval \
    --model hf \
    --model_args pretrained=EleutherAI/pythia-2.8b \
    --include_path ./ \
    --tasks demo_mmlu_high_school_geography_function_prompt \
    --limit 10 \
    --output output/demo_mmlu_high_school_geography_function_prompt/ \
    --log_samples

2023-11-29:11:59:08,312 INFO     [utils.py:160] NumExpr defaulting to 2 threads.
2023-11-29 11:59:09.348327: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-29 11:59:09.348387: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-29 11:59:09.348421: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-29 11:59:10.573752: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-11-29:11:59:14,044 INFO     [__main__.py:132] Verbosity set to INFO
2023-11-29:11:59:23,654 WARNING  [__main__.py:138]  --limit SHOULD ONLY BE USED FOR 

Next, we'll also show how to do this via preprocessing the dataset as necessary using the `process_docs` config field:

We will write a function that will modify each document in our evaluation dataset's split to add a field that is suitable for us to use in `doc_to_text`.

In [ ]:
YAML_mmlu_geo_string = """
include: mmlu_high_school_geography.yaml
task: demo_mmlu_high_school_geography_function_prompt_2
process_docs: !function utils_process_docs.process_docs
doc_to_text: "{{input}}"
doc_to_choice: "{{choices}}"
"""
with open("demo_mmlu_high_school_geography_process_docs.yaml", "w") as f:
    f.write(YAML_mmlu_geo_string)

DOC_TO_TEXT = """
def process_docs(dataset):
    def _process_doc(x):
        question = x["question"].strip()
        choices = x["choices"]
        option_a = choices[0]
        option_b = choices[1]
        option_c = choices[2]
        option_d = choices[3]
        doc["input"] = f"{question}\\nA. {option_a}\\nB. {option_b}\\nC. {option_c}\\nD. {option_d}\\nAnswer:"
        return out_doc

    return dataset.map(_process_doc)
"""

with open("utils_process_docs.py", "w") as f:
    f.write(DOC_TO_TEXT)

!lm_eval \
    --model hf \
    --model_args pretrained=EleutherAI/pythia-2.8b \
    --include_path ./ \
    --tasks demo_mmlu_high_school_geography_function_prompt_2 \
    --limit 10 \
    --output output/demo_mmlu_high_school_geography_function_prompt_2/ \
    --log_samples

We hope that this explainer gives you a sense of what can be done with and how to work with LM-Evaluation-Harnes v0.4.0 ! 

For more information, check out our documentation pages in the `docs/` folder, and if you have questions, please raise them in GitHub issues, or in #lm-thunderdome or #release-discussion on the EleutherAI discord server.